In [3]:
# !pip install -q rasterio
# !pip install -q gdal

     |████████████████████████████████| 19.3 MB 531 kB/s 


In [6]:
import numpy as np
import rasterio
from rasterio.plot import show
from rasterio.windows import Window
import cv2
import numpy as np
from matplotlib import pyplot as plt
import torch
import json
import glob
import os
from osgeo import gdal

In [7]:
class draw_bbox(): # Class For Showing Bounding Box Of Generated Geojson File.
    def __init__(self):
        pass


    def show1(self, img):
        plt.figure(figsize=(9,13))
        plt.axis("off")
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.show()


# ---------------------------------------------
    def saving_coordinates(self, json_path):
        data = json.load(open(json_path))
        final_data = []
        for i in data["features"]:
          tmp = {"id": "",
                  "boxes": ""}
          id_ = i["id"]
          box_coordinates = i["coordinates"][::-1]
          tmp["id"], tmp["boxes"] = id_, box_coordinates
          final_data.append(tmp)
        return final_data
    
    
    def read_coordinates(self, json_path):
        data = json.load(open(json_path))
        final_data = []
        for i in data["features"]:
            tmp = {"id": "",
                  "boxes": ""}
            id_ = i["properties"]["id"]
            box_coordinates = i["geometry"]["coordinates"][0][:4]
            tmp["id"], tmp["boxes"] = id_, box_coordinates
            final_data.append(tmp)
        return final_data
    
    
    
# ---------------------------------------------

    def get_all_boxes(self, final_list, start_x, end_x, start_y, end_y):
        
        fetched_bbox = []
        for i in final_list:
            tmp = {"id": "",
                "boxes": ""}
            if (i["boxes"][0][0] >= start_x and i["boxes"][2][0] <= end_x) and (i["boxes"][0][1] >= end_y and i["boxes"][2][1] <= start_y):
                tmp["id"], tmp["boxes"] = i["id"], i["boxes"]
                fetched_bbox.append(tmp)  
        return fetched_bbox


    def get_img_pixel_from_json(self, fetched_bbox_list, meta):
        converted_data = []
        for i in fetched_bbox_list:
            tmp = {"id": "",
                "boxes": ""}
            tmp["id"] = i["id"]
            dd = []
            for k in i["boxes"]:
                x_coord = k[0]
                y_coord = k[1]

                rowcol = rasterio.transform.rowcol(meta['transform'], xs=x_coord, ys=y_coord)
                y = rowcol[0]
                x = rowcol[1]
                dd.append([x,y])
            tmp["boxes"] = dd
            converted_data.append(tmp)
        return converted_data



    def convert_img_array(self, data, final_list, start_x, start_y, step_x, step_y):
        flag = False
        end_x = start_x + step_x
        end_y = start_y + step_y
        data = torch.from_numpy(data)
        data=data.permute(1, 2, 0)
        data = data.numpy()
        data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
    
        for i in final_list:
            if (i["boxes"][3][0] >= start_x) and (i["boxes"][1][0] <= end_x) and (i["boxes"][3][1] >= start_y) and (i["boxes"][1][1] <= end_y):
                x0 = i["boxes"][3][0] - start_x
                y0 = i["boxes"][3][1] - start_y
                x2 = i["boxes"][1][0] - start_x
                y2 = i["boxes"][1][1] - start_y
                bbox_width = x2 - x0
                bbox_height = y2 - y0
                cv2.rectangle(data, (x0,y0), (x0+bbox_width, y0+bbox_height), (255,0,0), 3)
                if flag == False:
                    flag = True
            else:
                pass
        return data, flag



    def crop_and_draw_bbox(self, height, width, crop_size, converted_data, dataset, saving_path, show_box):
        cnt = 0
        step = crop_size
        width_cut = crop_size
        height_cut = crop_size

        for col in range(0, height, step):
            if col + step >= height:
                height_cut = height - col

            for row in range(0, width, step):
                if row+width_cut >= width:
                    final_cut = width - row
                    data = dataset.read([1,2,3], window=Window(row, col, final_cut, height_cut))
                    final_img, flag = self.convert_img_array(data, converted_data, row, col, final_cut, height_cut)
                    if flag != False:
                        if show_box == True:
                            self.show1(final_img)
                            cv2.imwrite(f'{saving_path}/img_{cnt}.png', final_img)
                        else:
                            cv2.imwrite(f'{saving_path}/img_{cnt}.png', final_img)
                    cnt += 1

                else:
                    data = dataset.read([1,2,3], window=Window(row, col, width_cut, height_cut))
                    final_img, flag = self.convert_img_array(data, converted_data, row, col, width_cut, height_cut)
                    if flag != False:
                        if show_box == True:
                            self.show1(final_img)
                            cv2.imwrite(f'{saving_path}/img_{cnt}.png', final_img)
                        else:
                            cv2.imwrite(f'{saving_path}/img_{cnt}.png', final_img)
                    cnt += 1




    def main(self, tiff_file_path, geojson_file_path, img_saving_path, cropping_size = 1500, show_bbox = False):
        if not os.path.exists(img_saving_path):
            print("Not Found Image Saving Path... \n Created Image Saving Path...")
            os.makedirs(img_saving_path)
        cnt_ = 0
        tiff_file = tiff_file_path
        ds = gdal.Open(tiff_file)
        dataset = rasterio.open(tiff_file)
        meta_ = dataset.meta

        width = ds.RasterXSize
        height = ds.RasterYSize

        print("Height = ", height, "Width =", width)
        gt = ds.GetGeoTransform()

        minx = gt[0]
        miny = gt[3]
        maxy = gt[3] + width*gt[4] + height*gt[5] 
        maxx = gt[0] + width*gt[1] + height*gt[2]
        
        try:
            final_list = self.read_coordinates(geojson_file_path)
            print("Approaching First Format")
        except:
            final_list = self.saving_coordinates(geojson_file_path)
            print("Approaching Second Format")
        
        fetched_bbox = self.get_all_boxes(final_list, minx, maxx, miny, maxy)
        converted_data = self.get_img_pixel_from_json(fetched_bbox, meta_)
        self.crop_and_draw_bbox(height, width, cropping_size, converted_data, dataset, img_saving_path, show_bbox)

In [6]:
tiff_file1 = "tiff_files/NamarDaharat2_Orthomosaic_export_TueAug23092831065915.tif" # Tif File Path 
geo_data = "generated_geo_files/NamarDaharat2_Orthomosaic_export_TueAug23092831065915.geojson" # Generated Geojson File Path 


save_path = "saved_img" # Saving Path where images will be saved with drawed bounding boxes.



In [7]:
obj = draw_bbox() 
obj.main(tiff_file1, geo_data, save_path, cropping_size = 1500, show_bbox = True) # default cropping_size = 1500, show_bbox = True

Output hidden; open in https://colab.research.google.com to view.